# Deliverable Second Segment

In [5]:
# Initial imports
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

## Inspecting the Data Before Preprocessing

In [6]:
# Load and inspect the merged_spotify_songs.csv dataset
spotify_df = pd.read_csv("../Resources/merged_spotify_songs.csv")
spotify_df.head(10)

,id,name,artists,release_date,year,duration_ms,acousticness,danceability,energy,explicit,...,key,liveness,loudness,loudness_scaled,mode,popularity,speechiness,tempo,tempo_scaled,valence
0,02GDntOXexBFUvSgaXLPkd,Morceaux de fantaisie Op. 3: No. 2 Prélude in ...,['Sergei Rachmaninoff'],1921-01-01,1921,218773,0.993,0,0.088,0,...,1,0.3630,-21.091,0.609334,0,0.02,0.0456,92.867,0.380461,0.0731
1,08zfJvRLp7pjAb94MA9JmF,Il Etait Syndiqué,['Fortugé'],1921-01-01,1921,196560,0.982,1,0.257,0,...,8,0.5040,-16.415,0.682562,1,0.00,0.3990,109.378,0.448103,0.7710
2,0BMkRpQtDoKjcgzCpnqLNa,Dans La Vie Faut Pas S'en Faire,['Maurice Chevalier'],1921-01-01,1921,147133,0.995,0,0.260,0,...,9,0.2580,-16.894,0.675061,1,0.00,0.0557,85.146,0.348829,0.8260
3,0eQsdik7GTEy7M3UytCbSN,Morceaux de fantaisie Op. 3: No. 2 Prélude in ...,['Sergei Rachmaninoff'],1921-01-01,1921,218773,0.993,0,0.088,0,...,1,0.3630,-21.091,0.609334,0,0.00,0.0456,92.867,0.380461,0.0731
4,0H3k2CvJvHULnWChlbeFgx,La Vipère,['Georgel'],1921-01-01,1921,190800,0.990,0,0.363,0,...,5,0.2920,-12.562,0.742902,0,0.00,0.0546,174.532,0.715028,0.4930
5,0i7MdVu0tNEyUdgpCBilKe,Je M'donne,['Maurice Chevalier'],1921-01-01,1921,181733,0.996,1,0.274,0,...,2,0.3020,-14.001,0.720366,1,0.00,0.0410,79.218,0.324543,0.6780
6,0LcXzABeA84EgudqpNUN1I,Ud Taksimi,['Mehmet Kemiksiz'],1921-01-01,1921,184973,0.912,0,0.420,0,...,8,0.1080,-10.766,0.771028,0,0.00,0.1140,70.758,0.289884,0.2120
7,0NFeJgmTAV1kDfzSQNK41Z,10 Préludes Op. 23: No. 5 in G Minor. Alla marcia,['Sergei Rachmaninoff' 'Ruth Laredo'],1921-01-01,1921,221013,0.989,0,0.171,0,...,7,0.1160,-20.476,0.618965,0,0.03,0.0319,107.698,0.441221,0.2820
8,0Nk5f07H3JaEunGrYfbqHM,Come Back To Erin,['Phil Regan'],1921-01-01,1921,186467,0.957,0,0.212,0,...,2,0.2360,-13.300,0.731344,1,0.01,0.0358,85.726,0.351205,0.2180
9,0osXBirvQzPRfKSUDzHPCv,Korkma Sönmez,['Mehmet Kemiksiz'],1921-01-01,1921,155063,0.388,1,0.698,0,...,4,0.0768,-8.184,0.811463,0,0.00,0.0421,133.951,0.548775,0.7230


In [7]:
# find the dimensions of the dataset
spotify_df.shape

(169797, 21)

In [8]:
# Generate descriptive statistics
spotify_df.describe()

,year,duration_ms,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,loudness_scaled,mode,popularity,speechiness,tempo,tempo_scaled,valence
count,169797.000000,1.697970e+05,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000,169797.000000
mean,1977.221712,2.314078e+05,0.493309,0.601047,0.488541,0.084895,0.161962,5.200551,0.206696,-11.370476,0.761562,0.708570,0.315550,0.094066,116.946318,0.479110,0.532034
std,25.599875,1.213399e+05,0.376619,0.489684,0.267381,0.278727,0.309349,3.515181,0.176796,5.667187,0.088751,0.454423,0.215861,0.149961,30.728301,0.125889,0.262407
min,1921.000000,5.108000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1957.000000,1.710400e+05,0.094500,0.000000,0.263000,0.000000,0.000000,2.000000,0.098400,-14.470000,0.713022,0.000000,0.120000,0.034900,93.512000,0.383103,0.322000
50%,1978.000000,2.086000e+05,0.493000,1.000000,0.481000,0.000000,0.000204,5.000000,0.135000,-10.474000,0.775601,1.000000,0.330000,0.045000,114.773000,0.470206,0.544000
75%,1999.000000,2.629600e+05,0.888000,1.000000,0.710000,0.000000,0.086800,8.000000,0.263000,-7.118000,0.828158,1.000000,0.480000,0.075400,135.708000,0.555973,0.749000
max,2020.000000,5.403500e+06,0.996000,1.000000,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,1.000000,1.000000,0.969000,244.091000,1.000000,1.000000


In [9]:
# confirm data types
spotify_df.dtypes

id                   object
name                 object
artists              object
release_date         object
year                  int64
duration_ms           int64
acousticness        float64
danceability          int64
energy              float64
explicit              int64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
loudness_scaled     float64
mode                  int64
popularity          float64
speechiness         float64
tempo               float64
tempo_scaled        float64
valence             float64
dtype: object

In [6]:
# Find null values
for column in spotify_df.columns:
    print(f"Column {column} has {spotify_df[column].isnull().sum()}null values")

Column id has 0null values
Column name has 0null values
Column artists has 0null values
Column release_date has 0null values
Column year has 0null values
Column duration_ms has 0null values
Column acousticness has 0null values
Column danceability has 0null values
Column energy has 0null values
Column explicit has 0null values
Column instrumentalness has 0null values
Column key has 0null values
Column liveness has 0null values
Column loudness has 0null values
Column mode has 0null values
Column popularity has 0null values
Column speechiness has 0null values
Column tempo has 0null values
Column valence has 0null values


In [7]:
# Check for duplicates
print(f"Duplicated entries: {spotify_df.duplicated().sum()}")

Duplicated entries: 0


In [8]:
# set ID as index
spotify_df.set_index('id', inplace = True)
spotify_df.head()

,name,artists,release_date,year,duration_ms,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
id,,,,,,,,,,,,,,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],1928-01-01,1928,158648,0.995,1,0.1950,0,0.563,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790
6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",1928-01-01,1928,282133,0.994,0,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767
6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],1928-01-01,1928,104300,0.604,1,0.2200,0,0.000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800
6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],1928-09-25,1928,180760,0.995,1,0.1300,0,0.887,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200
6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1928-01-01,1928,687733,0.990,0,0.2040,0,0.908,11,0.0980,-16.829,1,1,0.0424,62.149,0.0693


In [9]:
# Create a new DataFrame that holds the string columns.
song_names_df = spotify_df[["name", "artists", "release_date", "year", "duration_ms"]].copy()
song_names_df 

,name,artists,release_date,year,duration_ms
id,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],1928-01-01,1928,158648
6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",1928-01-01,1928,282133
6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],1928-01-01,1928,104300
6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],1928-09-25,1928,180760
6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1928-01-01,1928,687733
...,...,...,...,...,...
4KppkflX7I3vJQk7urOJaS,Skechers (feat. Tyga) - Remix,"['DripReport', 'Tyga']",2020-05-15,2020,163800
1ehhGlTvjtHo2e4xJFB0SZ,Sweeter (feat. Terrace Martin),"['Leon Bridges', 'Terrace Martin']",2020-06-08,2020,167468
52eycxprLhK3lPcRLbQiVk,How Would I Know,"['Kygo', 'Oh Wonder']",2020-05-29,2020,180700


In [10]:
# Drop non numberical columns from main DataFrame
spotify_df = spotify_df.drop(columns = ["name", "artists", "release_date", "year", "duration_ms"])
spotify_df.head()

,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
id,,,,,,,,,,,,,
6KbQ3uYMLKb5jDxLF7wYDD,0.995,1,0.1950,0,0.563,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790
6KuQTIu1KoTTkLXKrwlLPV,0.994,0,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767
6L63VW0PibdM1HDSBoqnoM,0.604,1,0.2200,0,0.000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800
6M94FkXd15sOAOQYRnWPN8,0.995,1,0.1300,0,0.887,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200
6N6tiFZ9vLTSOIxkj8qKrd,0.990,0,0.2040,0,0.908,11,0.0980,-16.829,1,1,0.0424,62.149,0.0693


#### Split the data into input (X) and output (y)

In [11]:
# Separate the dataset into features (X) and target (y)
X = pd.get_dummies(spotify_df.drop('danceability', axis=1))
y = spotify_df["danceability"]

In [12]:
# Check the balance of our target values
y.value_counts()

1    102122
0     67787
Name: danceability, dtype: int64

#### Split the Data into Training and Testing

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, train_size=0.80)
X_train.shape

(135927, 12)

In [14]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(135927, 12)
(33982, 12)
(135927,)
(33982,)


In [15]:
Counter(y_train)

Counter({0: 54230, 1: 81697})

#### Random Oversampling

In [16]:
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [17]:
Counter(y_resampled)

Counter({0: 81697, 1: 81697})

#### Instantiate a Logistic Regression Model

In [18]:
model = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

#### Train the Logistic Regression Model

In [19]:
model.fit(X_resampled, y_resampled)

/Users/dianaescobar/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=200, random_state=1)

#### Make predictions

In [20]:
y_pred = model.predict(X_test)
results = pd.DataFrame({"Predictions": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(5)

,Predictions,Actual
0,0,0
1,1,1
2,1,0
3,1,1
4,0,1


#### Validate the Logistic Regression Model

In [21]:
# Calculating the accuracy score.
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.761633193148219

In [22]:
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [23]:
report = classification_report(y_test, y_pred)

In [24]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,10333,3224
Actual 1,4880,15545


Accuracy Score : 0.761633193148219
Classification Report
              precision    recall  f1-score   support

           0       0.68      0.76      0.72     13557
           1       0.83      0.76      0.79     20425

    accuracy                           0.76     33982
   macro avg       0.75      0.76      0.76     33982
weighted avg       0.77      0.76      0.76     33982

